In [1]:
    %%html
        <style>
        .box_style{
            /* width:40%; */
            height: auto;
            /* background-color:#242322; */
        }

        .select_box{
            width: 50%;
        }
        </style>

In [2]:
from he_demo import *
from IPython.display import display


demo = HeDemo()
display = display(*demo.createControls())


Unable to import ratio-db.py, perhaps due to a missing dependency. Using fallback code.


Output()